### Keras

In [1]:
import pandas as pd
import numpy as np
import pickle, time, math
import tensorflow as tf
from keras.models import Sequential
from keras import layers,losses,metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import Callback

Using TensorFlow backend.


In [2]:
with open('data.pickle','rb') as f:
    data=pickle.load(f)

In [3]:
#x_train=pd.concat([data['train_num'],data['train_cat']],axis=1)
#x_test=pd.concat([data['test_num'],data['test_cat']],axis=1)
#x_train=pd.concat([data['train_norm'],data['train_tbs']],axis=1)
#x_test=pd.concat([data['test_norm'],data['test_tbs']],axis=1)
x_train=pd.concat([data['train_norm'],data['train_tbs']],axis=1)
x_test=pd.concat([data['test_norm'],data['test_tbs']],axis=1)
y_train=data['y_train']
y_test=data['y_test']
column_names=x_train.columns.tolist()

In [4]:
x_train=np.array(x_train).astype('float32')
x_test=np.array(x_test).astype('float32')
y_train=np.array(y_train)
y_test=np.array(y_test)
#data=None

In [5]:
from sklearn.metrics import confusion_matrix
def confusion_mat(y_test,y_pred):
    true_negative, false_positive, false_negative, true_positive = confusion_matrix(y_test,y_pred).ravel()
    print("Actual\t\t|  Pos     |   Neg    |")
    print("Predicted\t-----------------")
    print("Pos\t\t| {0:08d} | {1:08d} |".format(true_positive,false_positive))
    print("Neg\t\t| {0:08d} | {1:08d} |\n".format(false_negative,true_negative))
    

In [6]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [7]:
class PostOpt(Callback):
    
    def __init__(self, x_train, y_train, x_test, y_test):
        #self.model = model

        self.epoch = []
        self.history = {}
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def on_epoch_end(self, epoch, logs={}):
        #tf.reset_default_graph()
        
        st=time.time()
        model=self.model
        test_pred=model.predict_classes(self.x_test)
        #test_au=auc(self.y_test,test_pred)
        #test_au=auc_roc(self.y_test,test_pred)
        #train_pred=model.predict_classes(self.x_train)
        #train_au=auc(self.y_train,train_pred)
        #train_au=auc_roc(self.y_train,train_pred)
        _,test_au=tf.metrics.auc(self.y_test,test_pred)
        #_,train_au=tf.metrics.auc(self.y_train,train_pred)
        sess=tf.Session()
        sess.run(tf.local_variables_initializer())
        #a0,a1=sess.run([train_au,test_au])
        a1=sess.run([test_au])
        tm=time.time()-st
        a0=0
        #print(a1)
        #print("Epoch {2:02d} Eval : {0:.4f} val_auc: {1:.4f} in {3:4.2f} sec".format(a0,a1[,epoch,tm))
        self.history.setdefault('chk_auc', []).append(a1)
        print(a0,a1)

In [15]:
def create_model(hidden_unit=32):

    input_feature_num=x_train.shape[1]
    
    model = Sequential(name="kdd2009")
    
    
    model.add(layers.BatchNormalization(name="Input_Norm", input_shape=(input_feature_num,)))
    #model.add(layers.Dropout(.7))
    #model.add(layers)
    
    model.add(layers.Dense(name='hidden_l1', units=hidden_unit))
    #model.add(layers.Dropout(.2))
    model.add(layers.Activation('relu'))
    
    model.add(layers.BatchNormalization(name="Input_Norm2"))
    model.add(layers.Dense(name='hidden_l2', units=hidden_unit))
    #model.add(layers.Dropout(.2))
    model.add(layers.Activation('relu'))
    
    model.add(layers.BatchNormalization(name="Input_Norm3"))         
    model.add(layers.Dense(name='hidden_l3', units=hidden_unit))
    #model.add(layers.Dropout(.2))
    model.add(layers.Activation('relu'))
    
    
    model.add(layers.Dense(name='output', units=1))
    model.add(layers.Activation('sigmoid'))
    
    return model

def fit(x_train,y_train,x_test, y_test, run_time, fold_n, batch_size=2000,epochs=20):
    model = None
    model = create_model()
    model.compile(optimizer='adam',loss=losses.binary_crossentropy, metrics=['accuracy',auc])
    filepath=run_time+"/model-{fold_n:02d}-{epoch:02d}-{loss:.4f}"
    #checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    earlystop=EarlyStopping(monitor='loss', min_delta=1e-4, patience=5, verbose=0, mode='auto')
    callbacks_list = [earlystop,SecondOpinion(model,x_train,y_train,x_test,y_test)]
    history=model.fit(x=x_train,y=y_train,batch_size=batch_size,epochs=epochs,
              validation_data=(x_test,y_test),verbose=1,callbacks=callbacks_list)
    return history

In [9]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

def generator(x, y, batch_size):
    #print("generator")
    idx_true = y[y==1]
    idx_false = y[y!=1]
    #sample_true=np.concatenate((x[idx_true,:],y_data[idx_true,:]),axis=1)
    #sample_false=np.concatenate((x[idx_false,:],y_data[idx_false,:]),axis=1)
    #print('positive:',idx_true.shape[0],' negative: ',idx_false.shape[0])
    x_true=x[idx_true]
    x_false=x[idx_false]
    y_true=y[idx_true]
    y_false=y[idx_false]

    number_samples_true_per_epoch = idx_true.shape[0]
    number_samples_false_per_epoch = idx_false.shape[0] 
    true_ratio=number_samples_true_per_epoch/(number_samples_false_per_epoch+number_samples_true_per_epoch)
    #false_ratio
    true_batches = int(math.ceil(batch_size*true_ratio))
    false_batches = int(batch_size*(1-true_ratio))
    number_of_batches=int(y.shape[0]/batch_size)
    counter=0
    #n_col=x.shape[1]
    #print(true_batches,false_batches)
    #print("\nblah")
    while 1:
    
        #print(counter,true_batches*counter,true_batches*(counter+1),
        #     false_batches*counter,false_batches*(counter+1),'\n')
        #,'number_of_true_batches',number_of_true_batches,'number_of_false_batches',number_of_false_batches,'\n')
        counter += 1
        #print("\n",counter,false_batches*(counter-1),false_batches*(counter),"\n")
        if number_of_batches <= counter:
                   
            x_batch = np.concatenate((x_true[true_batches*(counter-1):],
                                x_false[false_batches*(counter-1):])
                               ,axis=0).astype('float32')

            y_batch = np.concatenate((y_true[true_batches*(counter-1):],
                                y_false[false_batches*(counter-1):])
                               ,axis=0).astype('float32')
            counter = 0 
        else:
            x_batch = np.concatenate((x_true[true_batches*(counter-1):true_batches*(counter)],
                                x_false[false_batches*(counter-1):false_batches*(counter)])
                               ,axis=0).astype('float32')

            y_batch = np.concatenate((y_true[true_batches*(counter-1):true_batches*(counter)],
                                y_false[false_batches*(counter-1):false_batches*(counter)])
                               ,axis=0).astype('float32')            
        
        yield shuffle_in_unison(x_batch,y_batch)
        
def generator2(x, y, batch_size):
    idx_true = y[y==1]
    idx_false = y[y!=1]

    x_true=x[idx_true]
    #x_false=x[idx_false]
    y_true=y[idx_true]
    #y_false=y[idx_false]

    number_samples_true_per_epoch = idx_true.shape[0]
    number_samples_false_per_epoch = idx_false.shape[0] 
    true_ratio=number_samples_true_per_epoch/(number_samples_false_per_epoch+number_samples_true_per_epoch)
    #false_ratio
    #print('true ratio:',true_ratio)
    true_batches = int(math.ceil(batch_size*true_ratio))
    #false_batches = int(batch_size*(1-true_ratio))
    number_of_batches=int(y.shape[0]/batch_size)
    #print('number_of_batches:',number_of_batches)
    counter=0
    
    while 1:
        #print(counter)
        counter += 1
        idx=np.random.randint(0,y.shape[0]-1,[true_batches,])
        #_idx=idx[0:]
        #print(idx.shape)
        if number_of_batches <= counter:
            #print("2",number_of_batches)
            x_batch = np.concatenate((x_true[true_batches*(counter-1):true_batches*(counter)],
                                x[idx])
                               ,axis=0).astype('float32')

            y_batch = np.concatenate((y_true[true_batches*(counter-1):true_batches*(counter)],
                                y[idx])
                               ,axis=0).astype('float32')
            counter = 0 
        else:
            #print("1",number_of_batches)
            x_batch = np.concatenate((x_true[true_batches*(counter-1):true_batches*(counter)],
                                x[idx])
                               ,axis=0).astype('float32')

            y_batch = np.concatenate((y_true[true_batches*(counter-1):true_batches*(counter)],
                                y[idx])
                               ,axis=0).astype('float32')            
        #print(x_batch.shape)
        yield shuffle_in_unison(x_batch,y_batch)
        #yield x_batch,y_batch

In [10]:
batch_size=64
model = None
model = create_model()
model.compile(optimizer='adam',loss=losses.binary_crossentropy, metrics=['acc',auc])
filepath="weights-auc"
earlystop=EarlyStopping(monitor='val_auc', min_delta=2e-4, patience=10, verbose=0, mode='max')
checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
#callbacks_list = [earlystop,PostOpt(x_train,y_train,x_test,y_test),checkpoint]
callbacks_list = [earlystop]
#model.fit(x=x_train,y=y_train,batch_size=batch_size,epochs=10,
          #validation_data=(x_test,y_test),
#          verbose=1,callbacks=callbacks_list)

In [11]:
hist1=model.fit(x=x_train,y=y_train,batch_size=batch_size,epochs=100,
          validation_data=(x_test,y_test),
          verbose=1,callbacks=callbacks_list)

Train on 45000 samples, validate on 5000 samples
Epoch 1/100
45000/45000 [==============================] - 10s 219us/step - loss: 0.1280 - acc: 0.9736 - auc: 0.4942 - val_loss: 0.0893 - val_acc: 0.9822 - val_auc: 0.5018
Epoch 2/100
45000/45000 [==============================] - 8s 175us/step - loss: 0.1004 - acc: 0.9822 - auc: 0.4966 - val_loss: 0.0898 - val_acc: 0.9822 - val_auc: 0.4932
Epoch 3/100
45000/45000 [==============================] - 7s 159us/step - loss: 0.0974 - acc: 0.9822 - auc: 0.4941 - val_loss: 0.0899 - val_acc: 0.9822 - val_auc: 0.4946
Epoch 4/100
45000/45000 [==============================] - 8s 188us/step - loss: 0.0959 - acc: 0.9822 - auc: 0.4956 - val_loss: 0.0896 - val_acc: 0.9822 - val_auc: 0.4954
Epoch 5/100
45000/45000 [==============================] - 7s 152us/step - loss: 0.0946 - acc: 0.9822 - auc: 0.4968 - val_loss: 0.0902 - val_acc: 0.9822 - val_auc: 0.4952
Epoch 6/100
45000/45000 [==============================] - 7s 152us/step - loss: 0.0932 - acc: 

In [16]:
batch_size=64
model2 = None
model2 = create_model()
model2.compile(optimizer='adam',loss=losses.binary_crossentropy, metrics=['acc',auc])
filepath="weights-auc"
earlystop=EarlyStopping(monitor='acc', min_delta=1e-3, patience=10, verbose=0, mode='max')
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [earlystop,PostOpt(x_train,y_train,x_test,y_test),checkpoint]
callbacks_list = [earlystop,checkpoint]

In [17]:
hist=model2.fit_generator(generator=generator2(x_train, y_train,batch_size)
                    ,steps_per_epoch=x_train.shape[0]/batch_size
                    ,epochs=100
                    ,validation_data=(x_test,y_test)
                    ,callbacks=callbacks_list
                    ,shuffle=True
                    ,verbose=1
                   )

Epoch 1/100
704/703 [==============================] - 13s 18ms/step - loss: 0.1192 - acc: 0.9698 - auc: 0.4817 - val_loss: 0.1089 - val_acc: 0.9822 - val_auc: 0.4642

Epoch 00001: acc improved from -inf to 0.96712, saving model to weights-auc
Epoch 2/100
704/703 [==============================] - 10s 14ms/step - loss: 0.0753 - acc: 0.9870 - auc: 0.4685 - val_loss: 0.1054 - val_acc: 0.9820 - val_auc: 0.4733

Epoch 00002: acc improved from 0.96712 to 0.98729, saving model to weights-auc
Epoch 3/100
704/703 [==============================] - 9s 12ms/step - loss: 0.0886 - acc: 0.9833 - auc: 0.4806 - val_loss: 0.1023 - val_acc: 0.9820 - val_auc: 0.4961

Epoch 00003: acc did not improve from 0.98729
Epoch 4/100
704/703 [==============================] - 9s 12ms/step - loss: 0.0869 - acc: 0.9843 - auc: 0.5057 - val_loss: 0.1005 - val_acc: 0.9822 - val_auc: 0.5057

Epoch 00004: acc did not improve from 0.98729
Epoch 5/100
704/703 [==============================] - 8s 12ms/step - loss: 0.0775 

In [19]:
model2.save('model1')

In [20]:
model2.evaluate(x=x_train,y=y_train,batch_size=batch_size)

45000/45000 [==============================] - 2s 46us/step


[0.09775600823362668, 0.9822, 0.5222381877581278]

In [21]:
from keras.models import load_model
model3=load_model("model1",custom_objects={'auc': auc})

In [22]:
model3.evaluate(x=x_train,y=y_train,batch_size=batch_size)

45000/45000 [==============================] - 2s 54us/step


[0.09775600823362668, 0.9822, 0.5088837131023407]

In [23]:
pred=model2.predict_classes(x=x_train,batch_size=batch_size)
confusion_mat(y_train,pred)
pred=model3.predict_classes(x=x_train,batch_size=batch_size)
confusion_mat(y_train,pred)

Actual		|  Pos     |   Neg    |
Predicted	-----------------
Pos		| 00000000 | 00000000 |
Neg		| 00000801 | 00044199 |

Actual		|  Pos     |   Neg    |
Predicted	-----------------
Pos		| 00000000 | 00000000 |
Neg		| 00000801 | 00044199 |



In [24]:
model2.evaluate(x=x_test,y=y_test,batch_size=batch_size)

5000/5000 [==============================] - 1s 103us/step


[0.09791486065387726, 0.9822, 0.5399303087234497]

In [41]:
def evaluate_col(col,x,x_test,y_test,model):
    #global x,x_test,model
    x_col=x[:,col]
    #mu=np.mean(x_col)
    #sigma=np.std(x_col)    
    #x[:,col]=np.random.normal(mu,sigma,x_col.shape)
    np.random.shuffle(x_col)
    x[:,col]=x_col
    loss=model.evaluate(x,y_test,verbose=0)[0]
    x[:,col]=x_test[:,col]    
    return loss
    #losses_dict[proc_n]=loss

In [27]:
res1=model.evaluate(x=x_test,y=y_test,batch_size=batch_size,verbose=0)[0]
no_columns=x_test.shape[1]
x=x_test.copy()
loss_gains1=[]
for col in range(len(column_names)):
    #col=col+1000
    loss=evaluate_col(col,x,x_test,model)
    loss_gain=loss/res1
    #print("col:",col,"loss/res:",loss_gain)
    loss_gains1.append([loss_gain,loss_gain-1])

loss_gain1=pd.DataFrame(loss_gains1,columns=['loss','delta'],index=column_names)

In [42]:
res=model2.evaluate(x=x_train,y=y_train,batch_size=batch_size,verbose=0)[0]
no_columns=x_test.shape[1]
x=x_train.copy()
loss_gains2=[]
for col in range(len(column_names)):
    #col=col+1000
    loss=evaluate_col(col,x,x_train,y_train,model2)
    loss_gain=loss/res
    #print("col:",col,"loss/res:",loss_gain)
    loss_gains2.append([loss_gain,loss_gain-1])
    
loss_gain3=pd.DataFrame(loss_gains2,columns=['loss','delta'],index=column_names)
loss_gain3['ratio']=(loss_gain3['loss']/np.max(loss_gain3['loss'],axis=0))

In [45]:
loss_gain3.sort_values(['ratio'],ascending=False).head(5)

,loss,delta,ratio
Var113,1.004799,0.004799,1.000000
Var76,1.004783,0.004783,0.999984
Var198,1.002993,0.002993,0.998203
Var94,1.001832,0.001832,0.997047
Var133,1.001797,0.001797,0.997012


In [39]:
loss_gain2.sort_values(['ratio'],ascending=False).head(20)

,loss,delta,ratio
Var162,1.025390,0.025390,1.000000
Var124,1.011540,0.011540,0.986493
Var134,1.004572,0.004572,0.979698
Var163,1.003806,0.003806,0.978950
Var76,1.002900,0.002900,0.978067
Var153,1.001546,0.001546,0.976746
Var177,1.001128,0.001128,0.976339
Var150,1.001088,0.001088,0.976300
Var214,1.000936,0.000936,0.976152
Var192,1.000917,0.000917,0.976133


In [24]:
(np.abs(loss_gain2)).sort_values(['delta'],ascending=False).head(20)

,loss,delta
Var118,1.013408,0.013408
Var56,0.990226,0.009774
Var46,0.991621,0.008379
Var74,0.991960,0.008040
Var9,1.007326,0.007326
Var217,0.992894,0.007106
Var165,1.005965,0.005965
Var35,0.994079,0.005921
Var128,1.005786,0.005786
Var51,0.995246,0.004754


In [95]:
selected_columns=loss_gain2.loc[(loss_gain2.loss < 1.0)].index

In [90]:
_cols=(np.abs(loss_gain2)).sort_values(['delta'],ascending=False).head(50)
selected_columns=_cols.index

In [96]:
selected_column_id=[]
_=[selected_column_id.append(column_names.index(col)) for col in selected_columns]

In [98]:
def create_model(hidden_unit=16,input_feature_num=x_train.shape[1]):

    #input_feature_num=x_train.shape[1]
    
    model = Sequential(name="kdd2009")
    
    
    model.add(layers.BatchNormalization(name="Input_Norm", input_shape=(input_feature_num,)))
    #model.add(layers.Dropout(.7))
    #model.add(layers)
    
    model.add(layers.Dense(name='hidden_l1', units=hidden_unit))
    #model.add(layers.Dropout(.7))
    model.add(layers.Activation('relu'))
    
    model.add(layers.Dense(name='hidden_l2', units=hidden_unit))
    model.add(layers.Activation('relu'))
    
    model.add(layers.Dense(name='output', units=1))
    model.add(layers.Activation('sigmoid'))
    
    return model

In [99]:
batch_size=64
#model2 = None
model3 = create_model(input_feature_num=len(selected_column_id))
model3.compile(optimizer='adam',loss=losses.binary_crossentropy, metrics=['acc',auc])
filepath="weights-auc-{epoch:02d}-{loss:.4f}"
earlystop=EarlyStopping(monitor='val_auc', min_delta=1e-3, patience=5, verbose=0, mode='max')
checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
callbacks_list = [earlystop,PostOpt(x_train,y_train,x_test[:,selected_column_id],y_test),checkpoint]

In [100]:
hist=model3.fit_generator(generator=generator2(x_train[:,selected_column_id], y_train,batch_size)
                    ,steps_per_epoch=x_train.shape[0]/batch_size
                    ,epochs=100
                    ,validation_data=(x_test[:,selected_column_id],y_test)
                    ,callbacks=callbacks_list
                    ,shuffle=True
                    ,verbose=1
                   )

Epoch 1/100
704/703 [==============================] - 9s 13ms/step - loss: 0.1190 - acc: 0.9753 - auc: 0.4899 - val_loss: 0.1159 - val_acc: 0.9776 - val_auc: 0.5012
0 [0.4976583]

Epoch 00001: val_auc improved from -inf to 0.50123, saving model to weights-auc-01-0.1199
Epoch 2/100
704/703 [==============================] - 6s 9ms/step - loss: 0.0868 - acc: 0.9830 - auc: 0.5220 - val_loss: 0.1217 - val_acc: 0.9764 - val_auc: 0.5115
0 [0.49704745]

Epoch 00002: val_auc improved from 0.50123 to 0.51147, saving model to weights-auc-02-0.0830
Epoch 3/100
704/703 [==============================] - 6s 9ms/step - loss: 0.0703 - acc: 0.9869 - auc: 0.5244 - val_loss: 0.1172 - val_acc: 0.9792 - val_auc: 0.5207
0 [0.4984728]

Epoch 00003: val_auc improved from 0.51147 to 0.52067, saving model to weights-auc-03-0.0667
Epoch 4/100
704/703 [==============================] - 6s 9ms/step - loss: 0.0684 - acc: 0.9875 - auc: 0.5240 - val_loss: 0.1159 - val_acc: 0.9796 - val_auc: 0.5223
0 [0.49867645]

E

In [85]:
res=model3.evaluate(x=x_test[:,selected_column_id],y=y_test,batch_size=batch_size,verbose=0)[0]
no_columns=x_test.shape[1]
x=x_test[:,selected_column_id].copy()
loss_gains3=[]
for col in range(len(selected_columns)):
    #col=col+1000
    loss=evaluate_col(col,x,x_test[:,selected_column_id],model3)
    loss_gain=loss/res
    #print("col:",col,"loss/res:",loss_gain)
    loss_gains3.append([loss_gain,loss_gain-1])
    
loss_gain3=pd.DataFrame(loss_gains3,columns=['loss','delta'],index=selected_columns)

In [103]:
pred=model3.predict_classes(x=x_test[:,selected_column_id],batch_size=batch_size)
mat=confusion_matrix(y_test,pred)
mat

array([[4899,   12],
       [  89,    0]], dtype=int64)

In [105]:
true_negative, false_positive, false_negative, true_positive = confusion_matrix(y_test,pred).ravel()
print('true positive:',true_positive)
print('false positive:',false_positive)
print('true negative;',true_negative)
print('false negative',false_negative)

true positive: 0
false positive: 12
true negative; 4899
false negative 89


In [179]:
from sklearn.metrics import confusion_matrix
def confusion_mat(y_test,y_pred):
    true_negative, false_positive, false_negative, true_positive = confusion_matrix(y_test,y_pred).ravel()
    print("Actual\t\t|  Pos     |   Neg    |")
    print("Predicted\t-----------------")
    print("Pos\t\t| {0:08d} | {1:08d} |".format(true_positive,false_positive))
    print("Neg\t\t| {0:08d} | {1:08d} |\n".format(false_negative,true_negative))
    

In [180]:
pred=model3.predict_classes(x=x_train[:,selected_column_id],batch_size=batch_size)
confusion_mat(y_train,pred)
pred=model2.predict_classes(x=x_train,batch_size=batch_size)
confusion_mat(y_train,pred)
pred=model.predict_classes(x=x_train,batch_size=batch_size)
confusion_mat(y_train,pred)

Actual		|  Pos     |   Neg    |
Predicted	-----------------
Pos		| 00000004 | 00000136 |
Neg		| 00000797 | 00044063 |

Actual		|  Pos     |   Neg    |
Predicted	-----------------
Pos		| 00000000 | 00000001 |
Neg		| 00000801 | 00044198 |

Actual		|  Pos     |   Neg    |
Predicted	-----------------
Pos		| 00000000 | 00000000 |
Neg		| 00000801 | 00044199 |



In [70]:
loss_gain3.sort_values(['loss'],ascending=False).head(20)

,loss,delta
Var68,1.118528,0.118528
Var46,1.086111,0.086111
Var105,1.077477,0.077477
Var92,1.065804,0.065804
Var11,1.040183,0.040183
Var188,1.036347,0.036347
Var103,1.034272,0.034272
Var84,1.029994,0.029994
Var45,1.026106,0.026106
Var56,1.025040,0.025040


In [86]:
loss_gain3.sort_values(['loss'],ascending=False).head(20)

,loss,delta
Var62,1.024996,0.024996
Var99,1.011205,0.011205
Var34,1.008103,0.008103
Var166,1.006981,0.006981
Var111,1.005699,0.005699
Var158,1.005058,0.005058
Var118,1.004223,0.004223
Var135,1.004218,0.004218
Var41,1.004051,0.004051
Var197,1.003540,0.003540


In [71]:
loss_gain2.sort_values(['loss'],ascending=False).head(20)

,loss,delta
Var118,1.013408,-0.013408
Var9,1.007326,-0.007326
Var165,1.005965,-0.005965
Var128,1.005786,-0.005786
Var13,1.003509,-0.003509
Var140,1.003363,-0.003363
Var155,1.003241,-0.003241
Var54,1.003192,-0.003192
Var124,1.003093,-0.003093
Var147,1.002979,-0.002979


In [87]:
selected_columns3=loss_gain3.loc[(loss_gain3.loss < 1.0)].index

In [88]:
selected_columns3

Index(['Var2', 'Var8', 'Var13', 'Var14', 'Var15', 'Var20', 'Var23', 'Var24',
       'Var26', 'Var28',
       ...
       'Var165_imputed', 'Var167_imputed', 'Var169_imputed', 'Var175_imputed',
       'Var178_imputed', 'Var179_imputed', 'Var183_imputed', 'Var185_imputed',
       'Var209_imputed', 'Var230_imputed'],
      dtype='object', length=118)